In [ ]:
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, auc
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
from keras.layers import Input, Dense
from keras import regularizers
from keras.callbacks import ModelCheckpoint

In [ ]:
# Autoencoder 모델을 설계합니다.
input_dim = X_train.shape[1] 
encoding_dim = 128

input_layer = Input(shape=(input_dim, )) 
encoder = Dense(encoding_dim, activation="tanh", activity_regularizer=regularizers.l1(10e-5))(input_layer) 
encoder = Dense(encoding_dim,activation="relu")(encoder) 
decoder = Dense(encoding_dim,activation='tanh')(encoder) 
encoder = Dense(int(encoding_dim / 2),activation="relu")(encoder) 
decoder = Dense(int(encoding_dim / 2),activation='tanh')(encoder) 
encoder = Dense(int(encoding_dim / 4),activation="relu")(encoder) 
decoder = Dense(int(encoding_dim / 4),activation='tanh')(encoder) 
encoder = Dense(int(encoding_dim / 8),activation="relu")(encoder) 
decoder = Dense(int(encoding_dim / 8),activation='tanh')(encoder) 
decoder = Dense(input_dim, activation='relu')(decoder) 
autoencoder = Model(inputs=input_layer, outputs=decoder) 

In [ ]:
# 모델을 학습시킵니다.
nb_epoch = 50 
batch_size = 16 

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
with tf.device('/gpu:0'):
  autoencoder.compile(optimizer=Adam(learning_rate=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0), 
                      loss='mse', 
                      metrics= ['accuracy']) 

  history = autoencoder.fit(X_train, y_train, 
                            epochs=nb_epoch, 
                            batch_size=batch_size,
                            validation_data=(X_test, y_test), 
                            verbose=1)